# Notebook 2: Expériences sur les modèles

**Auteurs:**  

Akram Farihi, Sami Abloui, Amalya Mourih 

**Objectifs de notebook:**

Dans ce notebook, nous réalisons l'entraînement sur les modèles choisis avec accomagnement de MLFlows et le reste. Nous avons le code preprocessing.py qui permettra de faire le prétraiment des données. et nous allons effectue nos expéricences



In [1]:
# les lib importé 
from sys import path

path.append('..')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from data.preprocessing  import preprocess_data

print("Les bibliothèques sont importées avec succès.")

Les bibliothèques sont importées avec succès.


Preprocessing data avant de pouvoir l'entraineer

In [ ]:
from data.data_loader import load_data
df_raw = load_data()
print("Données chargées avec succès.")

print("\n Prétraitement des données...")
print("\n -------------------------------------------------------------------------")
df = preprocess_data(df_raw)
print("Prétraitement des données terminé.")
print("Aperçu des données prétraitées:")
df.head()

Données chargées avec succès.

 Prétraitement des données...

 -------------------------------------------------------------------------
   ID  Age  Gender  Country  Coffee_Intake  Caffeine_mg  Sleep_Hours  \
0   1   40    Male  Germany            3.5        328.1          7.5   
1   2   33    Male  Germany            1.0         94.1          6.2   
2   3   42    Male   Brazil            5.3        503.7          5.9   
3   4   53    Male  Germany            2.6        249.2          7.3   
4   5   32  Female    Spain            3.1        298.0          5.3   

  Sleep_Quality   BMI  Heart_Rate Stress_Level  Physical_Activity_Hours  \
0          Good  24.9          78          Low                     14.5   
1          Good  20.0          67          Low                     11.0   
2          Fair  22.7          59       Medium                     11.2   
3          Good  24.7          71          Low                      6.6   
4          Fair  24.1          76       Medium         

,Gender,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,Occupation,Smoking,Alcohol_Consumption,Age_Group,Continent
0,1,3.5,328.1,7.5,2,24.9,78,0,14.5,0,2,0,0,2,2
1,1,1.0,94.1,6.2,2,20.0,67,0,11.0,0,3,0,0,2,2
2,1,5.3,503.7,5.9,1,22.7,59,1,11.2,1,1,0,0,2,0
3,1,2.6,249.2,7.3,2,24.7,71,0,6.6,1,2,0,0,3,2
4,0,3.1,298.0,5.3,1,24.1,76,1,8.5,1,4,0,1,2,2


: 

In [3]:

X = df.drop(columns=['Sleep_Quality'])
y = df['Sleep_Quality']

# appliquer une répartition stratifiée car nos données sont déséquilibrées

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

Nous allons ensuite construire un pipeline afin d’automatiser la normalisation des données, l’entraînement des modèles et la définition des bonnes grilles de paramètres. Nous avons choisi d’utiliser des modèles de classification plutôt que des modèles de régression, car notre objectif est de prédire une classe. Trois approches seront donc évaluées :

- SVM, un classifieur performant et adapté aux jeux de données de taille moyenne.

- RandomForest, 

- MLP, un réseau de neurones adapté aux données tabulaires, contrairement aux CNN destinés aux images et aux RNN conçus pour les données séquentielles.

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

pipelines = {
    "rf": Pipeline([
        ("scaler", MinMaxScaler()),
        ("model", RandomForestClassifier())
    ]),
    "svm": Pipeline([
        ("scaler", StandardScaler()),
        ("model", SVC())
    ]),
    "mlp": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MLPClassifier(max_iter=300))
    ])
}

param_grids = {
    "rf": {
        "model__n_estimators": [100, 200, 300],      
        "model__max_depth": [None, 5, 10, 20],       
        "model__min_samples_split": [2, 5, 10],      
        "model__min_samples_leaf": [1, 2, 4],       
        "model__bootstrap": [True, False]            
    },
    "svm": {
        "model__C": [0.1, 1, 10],
        "model__kernel": ["rbf", "linear"]
    },
    "mlp": {
        "model__hidden_layer_sizes": [(50,), (100,), (50,50)],
        "model__activation": ["relu", "tanh"],
        "model__alpha": [0.0001, 0.001]
    }
}

In [5]:
import mlflow

from sklearn.model_selection import GridSearchCV

from src.utils.mlflow_run_manager import save_run

best_models = {}
best_scores = {}

experiment_name = "Sleep_Quality_Classification_29_11_19_44"

run_ids = {}
mlflow.set_tracking_uri("mlruns")
mlflow.set_experiment(experiment_name)

for name in pipelines:
    print("Recherche modèle :", name)

    with mlflow.start_run(run_name=name) as run:

        scoring = {
            "accuracy": "accuracy",
            "recall_weighted": "recall_weighted",
            "precision_weighted": "precision_weighted",
            "f1_weighted": "f1_weighted"
        }
   
        grid = GridSearchCV(
            estimator=pipelines[name],
            param_grid=param_grids[name],
            cv=3,
            scoring=scoring,
            refit="f1_weighted",
            n_jobs=-1
        )
        
        grid.fit(X_train, y_train)
        
        best_models[name] = grid.best_estimator_
        best_scores[name] = grid.best_score_
                
        # Log metrics
        mlflow.log_metric("best_cv_score", grid.best_score_)
        for metric_name, metric_value in scoring.items():
            if metric_name in grid.cv_results_:
                mlflow.log_metric(f"mean_test_{metric_name}", grid.cv_results_[f'mean_test_{metric_name}'][grid.best_index_])
        
        mlflow.sklearn.log_model(grid.best_estimator_, artifact_path="model")

        # Log parameters
        mlflow.log_params(grid.best_params_)

        print("Meilleurs paramètres :", grid.best_params_)
        print("Score CV :", grid.best_score_)
        print()

        save_run(experiment_name, run.info.run_id, grid.best_score_, name)




c:\Users\samim\Documents\code\PD\Projet\.pdenv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


Recherche modèle : rf


2025/12/01 15:46:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 15:46:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'model__bootstrap': True, 'model__max_depth': 5, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5, 'model__n_estimators': 200}
Score CV : 0.9915154335205845



FileNotFoundError: [Errno 2] No such file or directory: '.../data/experiments/exps.json'

In [ ]:
best_model_name = max(best_scores, key=best_scores.get)
best_model = best_models[best_model_name]

print("Meilleur modèle :", best_model_name)


Meilleur modèle : svm


In [ ]:
print(run_ids)
print(best_scores)

{'rf': '5d827001230b4f718b6a2f51934f5a38', 'svm': '3ddd850f06a344f6ac33f34a05a37bf3', 'mlp': '5edc4e957b2b476bba8ed228488cc89e'}
{'rf': np.float64(0.9916466994061023), 'svm': np.float64(0.9926078725256856), 'mlp': np.float64(0.9920201995846734)}
